<a href="https://colab.research.google.com/github/dmsophia/ponderadas_mod11/blob/main/ponderada_sem7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalação de bibliotecas e carregamento dos dados

In [1]:
!pip install plotly

In [46]:
import pandas as pd
import gdown
import plotly.express as px
import numpy as np
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
from collections import Counter
import plotly.io as pio

In [3]:
file_id = "1DcC4jMMdTK4DW2mjoHM56aaPr0IvDAkl"
url = f"https://drive.google.com/uc?id={file_id}"
output = "creditcard.csv"
gdown.download(url, output, quiet=False)

df = pd.read_csv(output)

df.to_parquet('creditcard.parquet')

df = pd.read_parquet('creditcard.parquet')

df.head()

Downloading...
From (original): https://drive.google.com/uc?id=1DcC4jMMdTK4DW2mjoHM56aaPr0IvDAkl
From (redirected): https://drive.google.com/uc?id=1DcC4jMMdTK4DW2mjoHM56aaPr0IvDAkl&confirm=t&uuid=b7aa6dd7-091f-4990-b7b7-70485e60ac35
To: /content/creditcard.csv
100%|██████████| 683M/683M [00:06<00:00, 98.3MB/s]


,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Análise descritiva

In [4]:
df.describe()

df.select_dtypes(include=['object']).nunique()

,0
ProductCD,5
card4,4
card6,4
P_emaildomain,59
R_emaildomain,60
M1,2
M2,2
M3,2
M4,3
M5,2


In [5]:
missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0]
print(missing_values.sort_values(ascending=False))

dist2    552913
D7       551623
D13      528588
D14      528353
D12      525823
          ...  
V294         12
V295         12
V297         12
V298         12
V302         12
Length: 374, dtype: int64


## Gráfico de Distribuição de Fraudes


In [6]:
fraud_count = df['isFraud'].value_counts().reset_index()
fraud_count.columns = ['Tipo de Transação', 'Contagem']

fig = px.bar(fraud_count, x='Tipo de Transação', y='Contagem',
             title='Distribuição de Transações Fraudulentas e Não Fraudulentas')
fig.show()


1. **Transações Não Fraudulentas (0)**: A grande maioria das transações no dataset não são fraudulentas, representando mais de 500.000 transações.
   
2. **Transações Fraudulentas (1)**: Uma pequena fração das transações são fraudulentas, com uma contagem muito menor em comparação com as transações não fraudulentas (praticamente imperceptível no gráfico).

Essa grande diferença indica um **desbalanceamento** significativo no dataset, onde as transações não fraudulentas superam amplamente as fraudulentas. Esse desbalanceamento pode afetar o desempenho do modelo LSTM, fazendo com que ele tenda a prever a maioria das transações como não fraudulentas. Será necessário aplicar técnicas para lidar com esse desbalanceamento, como amostragem ou uso de métricas adequadas, como o F1-score e AUC-ROC.

# Preparação dos dados

Na preparação dos dados para o modelo há:
- Preenchimento dos valores ausentes com a mediana de cada coluna, garantindo que os dados fiquem completos sem afetar muito a distribuição original.
- Em seguida, são selecionadas as colunas numéricas e aplicadas uma normalização usando o `StandardScaler`, o que ajusta os valores para que tenham média 0 e desvio padrão 1, essencial para o bom desempenho de redes neurais como o LSTM.
- Por fim, a função `create_sequences` transforma os dados em sequências temporais de comprimento 10, agrupando cada bloco consecutivo de 10 registros em uma nova sequência, que será usada como entrada para o LSTM, que processa dados em ordem temporal.

In [7]:
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns

df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

In [8]:
scaler = StandardScaler()

df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

In [9]:
def create_sequences(df, seq_length):
    sequences = []
    for i in range(len(df) - seq_length):
        sequences.append(df[i: i + seq_length])
    return np.array(sequences)

seq_length = 10
data_sequences = create_sequences(df[numeric_cols].values, seq_length)

# Modelo

### Arquitetura da Rede

1. **Modelo `Sequential`**:  
   O modelo utilizado é sequencial, o que significa que as camadas são empilhadas uma após a outra.

2. **Primeira Camada LSTM**:
   - **Unidades (neurônios):** 128  
     Cada unidade LSTM é como uma célula de memória que pode "lembrar" informações por longos períodos, o que é útil para sequências temporais.
   - **Input shape (entrada):**
     A entrada da camada é uma sequência de transações. O `seq_length` indica quantas transações consecutivas são usadas para analisar o padrão (neste caso, 10 transações). `len(numeric_cols)` indica o número de características ou variáveis de cada transação (por exemplo, o valor da transação, hora do dia, etc.).
   - **Return sequences:** `True`  
     Isso significa que a camada LSTM não só retorna o último estado de saída, mas devolve a sequência completa de saídas em cada passo temporal. Isso é importante porque outra camada LSTM vem a seguir e precisa dessas informações temporais completas para continuar aprendendo padrões ao longo do tempo.

3. **Camada Dropout**:
   - **Taxa de Dropout:** 0.2  
     Esta camada ajuda a evitar o *overfitting* (quando o modelo "decora" os dados de treinamento, mas não generaliza bem para novos dados). A taxa de 0.2 significa que, durante o treinamento, 20% dos neurônios são desativados de forma aleatória em cada passo. Isso força a rede a não depender demais de alguns neurônios específicos, promovendo a robustez.

4. **Segunda Camada LSTM**:
   - **Unidades (neurônios):** 64  
     Esta camada tem 64 unidades LSTM, uma redução em comparação à primeira camada. A ideia é que a rede já tenha extraído uma boa parte dos padrões complexos com 128 unidades na primeira camada, e agora a segunda camada refina isso com menos unidades.
   - **Return sequences:** `False`  
     Aqui, a rede retorna apenas o último estado de saída (não a sequência completa), pois essa camada está próxima da camada de saída, e não precisamos mais das informações de sequência completa.

5. **Segunda Camada Dropout**:
   - **Taxa de Dropout:** 0.2  
     Novamente, 20% dos neurônios são desativados de maneira aleatória para prevenir *overfitting* e garantir que o modelo não fique dependente de algumas unidades.

6. **Camada de Saída (`Dense`)**:
   - **Unidades:** 1  
     A última camada tem apenas um neurônio, que é responsável por emitir a previsão final. Como este é um problema de classificação binária (fraude ou não fraude), uma única saída é suficiente.
   - **Ativação:** `sigmoid`  
     A função de ativação `sigmoid` converte a saída em um valor entre 0 e 1, interpretado como a probabilidade da transação ser fraudulenta. Se o valor estiver mais próximo de 1, indica maior probabilidade de fraude; se for próximo de 0, indica transação não fraudulenta.

### Compilação do Modelo

- **Otimizador (`adam`)**:  
  O otimizador `adam` é amplamente usado em redes neurais porque ajusta automaticamente a taxa de aprendizado durante o treinamento, fazendo com que a rede aprenda de maneira mais eficiente. Ele é baseado em duas técnicas populares, AdaGrad e RMSProp, combinando suas vantagens.
  
- **Função de perda (`binary_crossentropy`)**:  
  Como estamos lidando com um problema de classificação binária (fraude ou não), a função de perda `binary_crossentropy` é apropriada. Ela calcula o erro entre as previsões do modelo e os valores reais, ajudando a rede a melhorar suas previsões a cada iteração.

- **Métrica (`accuracy`)**:  
  A métrica de desempenho usada é a acurácia, que mede a proporção de previsões corretas. Ela avalia o quanto o modelo está acertando ao prever fraudes ou não fraudes.

### Treinamento do Modelo

- **Divisão dos dados**:  
  Os dados são divididos em 80% para treinamento e 20% para teste. Isso garante que o modelo possa ser treinado em uma parte dos dados e, depois, avaliado em dados que ele nunca viu, verificando sua capacidade de generalização.

- **Épocas (`epochs`):** 10  
  O modelo passa por todo o conjunto de treinamento 10 vezes (chamadas de épocas). A cada época, o modelo ajusta seus pesos com base nos erros cometidos, melhorando suas previsões.

- **Batch size:** 64  
  Durante o treinamento, o modelo não processa todos os dados de uma vez. Em vez disso, ele processa blocos de 64 amostras de cada vez (chamados de lotes), o que torna o treinamento mais eficiente e permite que o modelo ajuste os pesos mais frequentemente.

- **Validação**:  
  O modelo também é avaliado continuamente em dados de teste durante o treinamento. Isso permite verificar se o modelo está realmente aprendendo, ou se está começando a decorar os dados de treinamento, indicando *overfitting*.


In [10]:
model = Sequential()
model.add(LSTM(units=128, input_shape=(seq_length, len(numeric_cols)), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=64, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

X_train, X_test, y_train, y_test = train_test_split(data_sequences, df['isFraud'][seq_length:], test_size=0.2, random_state=42)

history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/10
7382/7382 [==============================] - 210s 28ms/step - loss: -1.9907 - accuracy: 0.0000e+00 - val_loss: -3.9215 - val_accuracy: 0.0000e+00
Epoch 2/10
7382/7382 [==============================] - 204s 28ms/step - loss: -6.9742 - accuracy: 0.0000e+00 - val_loss: -8.6539 - val_accuracy: 0.0000e+00
Epoch 3/10
7382/7382 [==============================] - 200s 27ms/step - loss: -11.8866 - accuracy: 0.0000e+00 - val_loss: -12.5682 - val_accuracy: 0.0000e+00
Epoch 4/10
7382/7382 [==============================] - 198s 27ms/step - loss: -17.4684 - accuracy: 0.0000e+00 - val_loss: -18.1785 - val_accuracy: 0.0000e+00
Epoch 5/10
7382/7382 [==============================] - 198s 27ms/step - loss: -23.5707 - accuracy: 0.0000e+00 - val_loss: -20.3652 - val_accuracy: 0.0000e+00
Epoch 6/10
7382/7382 [==============================] - 199s 27ms/step - loss: -29.2490 - accuracy: 0.0000e+00 - val_loss: -26.1256 - val_accuracy: 0.0000e+00
Epoch 7/10
7382/7382 [============================

# Avaliação do modelo

In [18]:
from sklearn.metrics import classification_report, roc_auc_score
import numpy as np

y_pred = model.predict(X_test)

y_pred = y_pred.flatten()

y_pred_binary = np.where(y_pred > 0.5, 1, 0)

if isinstance(y_test, pd.Series):
    y_test = y_test.values

y_test = y_test.astype(int)

print(classification_report(y_test, y_pred_binary))

auc_roc = roc_auc_score(y_test, y_pred)
print(f"AUC-ROC: {auc_roc}")


3691/3691 [==============================] - 26s 7ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to cont

              precision    recall  f1-score   support

           0       0.97      0.98      0.97    113915
           1       0.00      0.00      0.00         0
           5       0.00      0.00      0.00      4191

    accuracy                           0.95    118106
   macro avg       0.32      0.33      0.32    118106
weighted avg       0.93      0.95      0.94    118106

AUC-ROC: 0.5855073470087565


In [19]:
fig_accuracy = go.Figure()
fig_accuracy.add_trace(go.Scatter(x=list(range(len(history.history['accuracy']))),
                                  y=history.history['accuracy'],
                                  mode='lines', name='Acurácia Treinamento'))
fig_accuracy.add_trace(go.Scatter(x=list(range(len(history.history['val_accuracy']))),
                                  y=history.history['val_accuracy'],
                                  mode='lines', name='Acurácia Validação'))
fig_accuracy.update_layout(title='Curva de Acurácia')
fig_accuracy.show()

fig_loss = go.Figure()
fig_loss.add_trace(go.Scatter(x=list(range(len(history.history['loss']))),
                              y=history.history['loss'],
                              mode='lines', name='Perda Treinamento'))
fig_loss.add_trace(go.Scatter(x=list(range(len(history.history['val_loss']))),
                              y=history.history['val_loss'],
                              mode='lines', name='Perda Validação'))
fig_loss.update_layout(title='Curva de Perda')
fig_loss.show()

#### 1. **Curva de Acurácia**:

O gráfico mostra uma linha completamente plana na marca de 0% para ambos, tanto o conjunto de treinamento quanto o de validação. Isso indica que o modelo não está conseguindo aprender absolutamente nada sobre as classes, tanto para os dados de treinamento quanto para os de validação.
  - O modelo pode estar completamente dominado pelo desbalanceamento das classes, ignorando as transações fraudulentas (classe minoritária) e classificando todas as transações como não fraudulentas.
  - O dataset pode ter algum problema estrutural (como rótulos malformados, classes inesperadas como a classe "5").
  - Problemas na implementação do modelo ou na forma como os dados estão sendo passados.

#### 2. **Curva de Perda**:

O gráfico mostra uma queda contínua na função de perda tanto para o treinamento quanto para a validação, com perdas negativas ao longo do tempo. A perda negativa é anormal para uma função de perda como a entropia cruzada binária (`binary_crossentropy`), que normalmente deve ser positiva.

- A perda está diminuindo em ambas as curvas, mas isso não reflete um aprendizado real, uma vez que a acurácia é 0%.
- A perda negativa pode estar sendo causada por problemas nos dados ou pela forma como o modelo foi implementado. Geralmente, a perda negativa significa que algo não está sendo tratado corretamente nos cálculos da função de perda.

#### 3. **Resultados do Relatório de Classificação**:

No relatório de classificação anterior:

- **Classe 0 (Transações não fraudulentas)**: O modelo conseguiu capturar a maioria das transações não fraudulentas (classe majoritária).
- **Classe 1 (Transações fraudulentas)**: Nenhuma transação fraudulenta foi identificada, levando a um recall, precisão e F1-score de 0.
- **Classe 5**: A presença da classe "5" no relatório é inesperada e sugere problemas com os rótulos ou com os dados de entrada.

### Próximas Ações Recomendadas:

1. **Tratar o Desbalanceamento de Classes**:
   - **Ajuste de `class_weight`**: Atribuir mais peso à classe minoritária (fraudulenta) durante o treinamento. Isso ajudará o modelo a considerar mais as fraudes, já que atualmente ele está ignorando essa classe devido ao desbalanceamento extremo.

   - **Oversampling ou Undersampling**: Tente aumentar o número de transações fraudulentas ou diminuir o número de transações não fraudulentas no conjunto de dados de treinamento.

2. **Revisar os Rótulos dos Dados**:
   - Verifique se os rótulos estão corretos e se os dados foram processados adequadamente. A presença da classe "5" indica que algo está errado com os rótulos.
   
3. **Ajustar a Função de Perda**:
   - Verificar como os dados estão sendo passados para a função de perda. Se houver inconsistências nos rótulos (por exemplo, rótulos que não sejam 0 ou 1), isso pode causar problemas com a função de perda. A perda negativa precisa ser investigada a fundo..

# Segundo modelo

In [34]:
file_id = "1DcC4jMMdTK4DW2mjoHM56aaPr0IvDAkl"
url = f"https://drive.google.com/uc?id={file_id}"
output = "creditcard.csv"
gdown.download(url, output, quiet=False)

df = pd.read_csv(output)

df.to_parquet('creditcard.parquet')

df = pd.read_parquet('creditcard.parquet')

df.head()

Downloading...
From (original): https://drive.google.com/uc?id=1DcC4jMMdTK4DW2mjoHM56aaPr0IvDAkl
From (redirected): https://drive.google.com/uc?id=1DcC4jMMdTK4DW2mjoHM56aaPr0IvDAkl&confirm=t&uuid=67c19ff0-e2dd-4121-9a02-e4f9e67a8ba5
To: /content/creditcard.csv
100%|██████████| 683M/683M [00:03<00:00, 206MB/s]


,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Abaixo, são selecionadas as colunas numéricas do DataFrame e remove a coluna 'isFraud' (que não deve ser alterada). Ele preenche os valores ausentes nas colunas numéricas com a mediana de cada uma e, em seguida, padroniza esses dados, ajustando-os para ficarem na mesma escala usando `StandardScaler`.

In [35]:
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns

if 'isFraud' in numeric_cols:
    numeric_cols = numeric_cols.drop('isFraud')

df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

scaler = StandardScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

Valores únicos em 'isFraud': (array([0, 1]), array([569877,  20663]))
Número de valores nulos em 'isFraud': 0
Total de amostras: 590540
   isFraud
0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
Número de amostras após a filtragem: 590540


Abaixo são criadas sequências dos dados para serem usadas no LSTM. A função `create_sequences` gera subconjuntos consecutivos de linhas com um comprimento definido (`seq_length`). Aqui, o comprimento é 10. As sequências são extraídas das colunas numéricas e os rótulos de 'isFraud' começam a partir do 10º registro. Em seguida, ele verifica se os rótulos contêm apenas os valores esperados (0 ou 1). Se houver valores inesperados, um erro é gerado.

In [36]:
def create_sequences(df, seq_length):
    sequences = []
    for i in range(len(df) - seq_length):
        sequences.append(df[i: i + seq_length])
    return np.array(sequences)

seq_length = 10
data_sequences = create_sequences(df[numeric_cols].values, seq_length)
rótulos = df['isFraud'][seq_length:].values

print("Valores únicos após a filtragem:", np.unique(rótulos))
assert np.all(np.isin(rótulos, [0, 1])), "Rótulos contêm valores inesperados."

Valores únicos após a filtragem: [0. 1.]


In [37]:
if len(rótulos) > 0:
    X_train, X_test, y_train, y_test = train_test_split(data_sequences, rótulos, test_size=0.2, random_state=42)
else:
    raise ValueError("Nenhuma amostra disponível após a filtragem.")

assert len(X_train) == len(y_train), "Número de sequências de treino não corresponde ao número de rótulos de treino."
assert len(X_test) == len(y_test), "Número de sequências de teste não corresponde ao número de rótulos de teste."


Primeiramente, conta-se a distribuição das classes no conjunto de treinamento antes de aplicar o undersampling. Ele separa os índices de transações fraudulentas e não fraudulentas. Em seguida, subamostra aleatoriamente os índices da classe não fraudulenta para igualar o número de exemplos da classe fraudulenta. Esses índices são combinados e usados para criar um novo conjunto de treinamento balanceado. Por fim, ele imprime a nova distribuição das classes após o undersampling.

In [38]:
counter = Counter(y_train)
print(f"Distribuição das classes no treino antes do undersampling: {counter}")

fraudulent_indices = np.where(y_train == 1)[0]
non_fraudulent_indices = np.where(y_train == 0)[0]

non_fraudulent_indices_undersampled = np.random.choice(non_fraudulent_indices, size=len(fraudulent_indices), replace=False)

undersampled_indices = np.concatenate([fraudulent_indices, non_fraudulent_indices_undersampled])

X_train_undersampled = X_train[undersampled_indices]
y_train_undersampled = y_train[undersampled_indices]

counter_undersampled = Counter(y_train_undersampled)
print(f"Distribuição das classes no treino após o undersampling: {counter_undersampled}")


Distribuição das classes no treino antes do undersampling: Counter({0.0: 455952, 1.0: 16472})
Distribuição das classes no treino após o undersampling: Counter({1.0: 16472, 0.0: 16472})


A arquitetura da rede LSTM se mantém quase a mesma, apenas com a modificação da taxa de dropout para 30%.

In [39]:
model = Sequential()
model.add(LSTM(units=128, input_shape=(seq_length, len(numeric_cols)), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(units=64, return_sequences=False))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train_undersampled, y_train_undersampled, epochs=10, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/10
515/515 [==============================] - 38s 68ms/step - loss: 0.6770 - accuracy: 0.5580 - val_loss: 0.6352 - val_accuracy: 0.8750
Epoch 2/10
515/515 [==============================] - 31s 61ms/step - loss: 0.6655 - accuracy: 0.5801 - val_loss: 0.6666 - val_accuracy: 0.7354
Epoch 3/10
515/515 [==============================] - 32s 61ms/step - loss: 0.6569 - accuracy: 0.5967 - val_loss: 0.6141 - val_accuracy: 0.8127
Epoch 4/10
515/515 [==============================] - 31s 60ms/step - loss: 0.6432 - accuracy: 0.6170 - val_loss: 0.7015 - val_accuracy: 0.6274
Epoch 5/10
515/515 [==============================] - 31s 60ms/step - loss: 0.6206 - accuracy: 0.6444 - val_loss: 0.6738 - val_accuracy: 0.6867
Epoch 6/10
515/515 [==============================] - 31s 60ms/step - loss: 0.5859 - accuracy: 0.6797 - val_loss: 0.7656 - val_accuracy: 0.5706
Epoch 7/10
515/515 [==============================] - 31s 60ms/step - loss: 0.5387 - accuracy: 0.7156 - val_loss: 0.7464 - val_accuracy:

In [40]:
y_pred = model.predict(X_test)
y_pred_binary = np.where(y_pred > 0.5, 1, 0)
print(classification_report(y_test, y_pred_binary))

auc_roc = roc_auc_score(y_test, y_pred)
print(f"AUC-ROC: {auc_roc}")

3691/3691 [==============================] - 26s 7ms/step
              precision    recall  f1-score   support

         0.0       0.97      0.60      0.74    113915
         1.0       0.04      0.50      0.08      4191

    accuracy                           0.59    118106
   macro avg       0.51      0.55      0.41    118106
weighted avg       0.94      0.59      0.72    118106

AUC-ROC: 0.5674806916747221


In [45]:
pio.renderers.default = 'colab'

def plot_learning_curves(history):
    loss_train = history.history['loss']
    loss_val = history.history['val_loss']
    acc_train = history.history['accuracy']
    acc_val = history.history['val_accuracy']
    epochs = range(1, len(loss_train) + 1)

    fig_loss = go.Figure()
    fig_loss.add_trace(go.Scatter(x=list(epochs), y=loss_train, mode='lines', name='Perda Treino'))
    fig_loss.add_trace(go.Scatter(x=list(epochs), y=loss_val, mode='lines', name='Perda Validação'))

    fig_loss.update_layout(title='Curva de Perda - Treinamento vs Validação',
                           xaxis_title='Épocas',
                           yaxis_title='Perda',
                           hovermode='x unified')

    fig_loss.show()

    fig_acc = go.Figure()
    fig_acc.add_trace(go.Scatter(x=list(epochs), y=acc_train, mode='lines', name='Acurácia Treino'))
    fig_acc.add_trace(go.Scatter(x=list(epochs), y=acc_val, mode='lines', name='Acurácia Validação'))

    fig_acc.update_layout(title='Curva de Acurácia - Treinamento vs Validação',
                          xaxis_title='Épocas',
                          yaxis_title='Acurácia',
                          hovermode='x unified')

    fig_acc.show()

plot_learning_curves(history)

#### 1. **Curva de Perda (Loss)**:
- A perda no conjunto de treinamento está diminuindo continuamente, o que indica que o modelo está aprendendo com os dados de treinamento.
- No entanto, a perda no conjunto de validação está aumentando ao longo das épocas, especialmente após a 4ª época. Isso é um sinal clássico de overfitting, onde o modelo está se ajustando muito bem aos dados de treinamento, mas está generalizando mal nos dados de validação.

#### 2. **Curva de Acurácia (Accuracy)**:
- A acurácia de treinamento está aumentando constantemente, o que é esperado à medida que o modelo aprende.
- A acurácia de validação está flutuando e não melhora significativamente. Ela cai drasticamente em algumas épocas, o que reforça o problema de overfitting.
  
#### 3. **Métricas de Classificação**:
  - Para a classe não fraudulenta (`0.0`), a precisão é alta (`0.97`), mas o recall é relativamente baixo (`0.60`), o que indica que o modelo está identificando muitas transações não fraudulentas como fraudulentas.
  - Para a classe fraudulenta (`1.0`), a precisão é extremamente baixa (`0.04`), mas o recall é surpreendentemente alto (`0.50`), o que indica que o modelo está identificando metade das fraudes, mas ao custo de muitos falsos positivos.
  - O F1-score para a classe fraudulenta é muito baixo (`0.08`), indicando que o modelo está longe de ser eficaz para detectar fraudes de forma consistente.
  
#### 4. **Acurácia Geral**:
- A acurácia geral é 59%, o que parece ser influenciado pela classe majoritária (não fraudulenta), que representa a maior parte do conjunto de dados.

#### 5. **AUC-ROC**:
- O valor AUC-ROC de 0.567 está muito próximo do valor de 0.5, o que sugere que o modelo tem pouco poder discriminatório para distinguir entre fraudes e não fraudes. O modelo está quase equivalente a fazer previsões aleatórias.

### Conclusões:

1. **Overfitting**: O gráfico de perda mostra claramente que o modelo está se ajustando bem aos dados de treinamento, mas tem um desempenho ruim nos dados de validação, indicando que o modelo está superajustado aos dados de treino.
   
2. **Baixo Desempenho na Detecção de Fraudes**: O modelo tem uma precisão muito baixa para detectar fraudes. Ele está identificando apenas uma pequena fração das fraudes corretamente, e mesmo assim, isso vem com uma grande quantidade de falsos positivos.

3. **Desequilíbrio de Classes**: O problema de desequilíbrio de classes é evidente. A maioria das transações não são fraudulentas, o que provavelmente está influenciando o modelo a prever mais transações como não fraudulentas.

### Próximos Passos:

1. **Técnicas de Balanceamento**:
   -*Undersampling: Continuar explorando o undersampling para a classe não fraudulenta, mas de uma forma mais cuidadosa.
   -Oversampling ou SMOTE: Tentar aumentar a quantidade de exemplos da classe fraudulenta, utilizando técnicas como SMOTE (Synthetic Minority Over-sampling Technique), que cria exemplos sintéticos da classe minoritária.
   
2. **Regularização**:
   - Tente adicionar camadas de Dropout com uma taxa mais alta para reduzir o overfitting.
   - Aumentar a regularização L2 pode ajudar a evitar que o modelo superajuste os dados de treinamento.

3. **Tuning dos Hiperparâmetros**:
   - Ajustar hiperparâmetros como o número de épocas, tamanho do lote (batch size), e taxa de aprendizado para encontrar o equilíbrio certo entre a complexidade do modelo e a capacidade de generalização.
   
4. **Ajuste da Arquitetura**:
   - Reduzir o número de neurônios nas camadas LSTM ou adicionar mais camadas intermediárias para melhorar a capacidade de generalização.

# Discussão

### Comparação entre os Modelos:

#### **Modelo 1:**
O modelo 1 apresentou um desempenho extremamente insatisfatório, sem sinais de aprendizado real, com os principais problemas sendo:

1. **Curva de Acurácia**:
   - A acurácia tanto no conjunto de treinamento quanto no de validação ficou completamente plana e estagnada em 0%. Isso indica que o modelo não conseguiu aprender nada útil, muito provavelmente devido a problemas no processamento dos dados ou forte desbalanceamento das classes.
   
2. **Curva de Perda**:
   - Embora a perda estivesse diminuindo, houve um comportamento anormal de perdas negativas, o que não é comum em uma função de perda como a entropia cruzada binária. Isso geralmente aponta para erros graves nos rótulos dos dados ou na implementação do modelo.
   
3. **Relatório de Classificação**:
   - O modelo conseguiu identificar apenas a classe majoritária (transações não fraudulentas), enquanto ignorou completamente as transações fraudulentas (classe minoritária). O recall, precisão e F1-score da classe 1 (fraudes) foram 0, sugerindo que o modelo não aprendeu a distinguir fraudes.
   - A presença de uma classe inesperada ("5") no relatório foi um grande alerta, sugerindo que havia problemas no pré-processamento dos rótulos ou que as classes estavam malformadas.

#### **Modelo 2:**
O modelo 2 teve um desempenho melhor comparado ao modelo 1, mas ainda apresentou sinais de overfitting e desafios relacionados ao desbalanceamento de classes.

1. **Curva de Perda**:
   - A perda no conjunto de treinamento está diminuindo, o que indica que o modelo está aprendendo com os dados de treinamento.
   - No entanto, a perda no conjunto de validação está aumentando após a 4ª época, indicando overfitting. Isso significa que o modelo está ajustando-se muito aos dados de treinamento e não está generalizando bem nos dados de validação.
   
2. **Curva de Acurácia**:
   - A acurácia no conjunto de treinamento está aumentando continuamente, o que é esperado.
   - A acurácia no conjunto de validação está flutuando significativamente, mostrando que o modelo está com dificuldade de generalizar. Isso também é um sinal de overfitting.
   
3. **Relatório de Classificação**:
   - Para a classe não fraudulenta (`0.0`), a precisão é alta (`0.97`), mas o recall é baixo (`0.60`), indicando que muitas transações não fraudulentas estão sendo classificadas incorretamente.
   - Para a classe fraudulenta (`1.0`), a precisão é muito baixa (`0.04`), mas o recall é alto (`0.50`), indicando que o modelo está detectando metade das fraudes, mas com muitos falsos positivos.
   - O F1-score para fraudes é extremamente baixo (`0.08`), o que demonstra que o modelo não consegue identificar fraudes de forma eficaz.

### Estratégias para Melhorar a Performance:

Entre o modelo 1 e o modelo 2, várias melhorias foram aplicadas para tentar resolver os problemas encontrados inicialmente:

#### 1. **Revisão dos Rótulos**:
   - Um dos primeiros ajustes críticos foi corrigir os rótulos incorretos no modelo 1. O modelo estava tentando prever classes inesperadas como "5", o que foi um sinal de dados malformados. O modelo 2 passou a usar rótulos adequados, focando corretamente nas classes `0` e `1`.
   
#### 2. **Tratar o Desbalanceamento de Classes**:
   - No modelo 2, foi aplicada a técnica de undersampling para equilibrar o número de transações fraudulentas e não fraudulentas, já que o desbalanceamento extremo do modelo 1 foi uma das razões para seu desempenho ruim.
   - O undersampling ajudou a melhorar a sensibilidade do modelo em detectar fraudes, mas isso veio ao custo de falsos positivos, o que comprometeu a precisão da classe fraudulenta.

#### 3. **Redução de Overfitting**:
   - No modelo 2, o overfitting foi identificado como um problema chave. A adição de camadas de dropout foi um passo importante para mitigar isso, mas não foi suficiente para eliminar completamente o overfitting.

### Próximos Passos:

1. **Aplicar SMOTE ou Oversampling**:
   - Utilizar técnicas como o SMOTE para gerar mais exemplos da classe minoritária e reequilibrar o dataset, preservando os dados da classe majoritária e evitando o descarte de amostras úteis.

2. **Mais Regularização e Ajuste da Arquitetura**:
   - Aumentar o uso de Dropout e adicionar regularização L2 para combater o overfitting.
   - Ajustar a arquitetura do modelo reduzindo o número de neurônios ou adicionando camadas intermediárias para melhorar a capacidade de generalização.

3. **Ajustar Hiperparâmetros**:
   - Ajustar o número de épocas, tamanho do lote, e taxa de aprendizado para melhorar o aprendizado sem overfitting.

4. **Monitorar as Métricas de Desempenho**:
   - Continuar monitorando a curva de perda e acurácia para avaliar se os ajustes estão melhorando o desempenho do modelo sem comprometer sua capacidade de generalização.